In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18396 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

Setting up unattended-upgrades (0.98ubuntu1.1) ...

Creating config file /etc/apt/apt.conf.d/20auto-upgrades with new version

Creating config file /etc/apt/apt.conf.d/50unattended-upgrades with new version
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up dirmngr (2.1.15-1ubuntu8.1) ...
Setting up cron (3.0pl1-128ubuntu5) ...
Adding group `crontab' (GID 102) ...
Done.
update-rc.d: warning: start and stop actions are no longer supported; falling back to defaults
update-rc.d: warning: stop runlevel arguments (1) do not match cron Default-Stop values (none)
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Setting up kmod (24-1ubuntu2) ...
Setting up libdbus-glib-1-2:amd64 (0.108-2) ...
Setting up python3-gi (3.24.1-2build1) ...
Setting up module-init-tools (24-1ubuntu2) ...
Setting up python3-software-properties (0.96.24.1

In [0]:
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

In [3]:
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p MyDrive
!google-drive-ocamlfuse MyDrive

In [5]:
% cd MyDrive


/content/MyDrive


In [0]:
!ls


The objective of this experiment is to learn how to do Data preprocessing.

We will be using district wise demographics, enrollments, school and teacher indicator data to predict whether the literacy rate is high / medium / low in each district.

Data preprocessing is an important step of solving every machine learning problem. Most of the datasets used with Machine Learning problems need to be processed / cleaned / transformed so that a Machine Learning algorithm can be trained on it.

1. Data Cleaning → In this step the primary focus is on
    -Handling missing data
    -Handling nosiy data
    -Detection and removal of outliers

2. Data Integration → This process is used when data is gathered from various data sources
and data are combined to form consistent data. This data after performing cleaning is used
for analysis.

3. Data Transformation → In this step we will convert the raw data into a specified for-
mat according to the need of the model we are building. There are many options used for
transforming the data as below:
    
    -Normalization
    
    -Aggregation
    
    -Generalization

4. Data Reduction → After data transformation and scaling the redundancy within the data
is removed and efficiently organizing the data is performed.

# **GROUP LAB 2**

We have four different files

Districtwise_Basicdata.csv
Districtwise_Enrollment_details_indicator.csv
Districtwise_SchoolData.csv
Districtwise_Teacher_indicator.csv
These files contain the neccesary data to solve the problem. Load all the files correctly, after observing the header level details, data records etc

In [0]:

##If you wish to change the data files 
  DIR = "colab_docs/CSV"
  basicData = DIR +"/Districtwise_Basicdata.csv"
  enrollmentData = DIR + "/Districtwise_Enrollment_details_indicator.csv"
  schoolData = DIR + "/Districtwise_SchoolData.csv"
  teacherData = DIR + "/Districtwise_Teacher_indicator.csv"

In [0]:
def change_to_num(s):
  s = s.strip().lower()
  if s == 'high':
    return int('2')
  if s == 'medium':
    return int('1')
  return int('0')

In [0]:

import pandas as pd
basicdata = pd.read_csv(basicData,header=1,converters={'overall_lit': change_to_num} )
enrollmentdata = pd.read_csv(enrollmentData,header=3)
schooldata = pd.read_csv(schoolData, header=3)
teacherdata = pd.read_csv(teacherData,header=3)




In [0]:
# 
# import os
# files = []
# for f in os.walk(DIR):
#     files.append(f)

# list = files[0][-1]
# list


Remove the unwanted columns, which are unlikely to contribute for the prediction of overall literacy category(Manual Feature Selection)

The data is present in different files. We need to integrate all of the data into a single data frame.

For example all the data of the Anantapur district (AP) should form a single row.
Recall your SQL: create or identify a unique identifier for each row in all the data frames and use it to combine the data.
One possibility -- by no means the best-- is (year, state code, district code) combination

In [0]:
SubSchooldata=schooldata[['distcd','schgovt1','schgovt2','schgovt3','schgovt4','schgovt5','schgovt6','schgovt7','schgovt9','schpvt1','schpvt2','schpvt3','schpvt4','schpvt5','schpvt6','schpvt7','schpvt9']]
SubEnrolldata = enrollmentdata[['distcd','Enr Govt1','Enr Govt2','Enr Govt3','Enr Govt4','Enr Govt5','Enr Govt6','Enr Govt7','Enr Govt9','Enr Pvt1','Enr Pvt2','Enr Pvt3','Enr Pvt4','Enr Pvt5','Enr Pvt6','Enr Pvt7','Enr Pvt9']]
SubBasicdata = basicdata[['distcd', 'totpopulation','overall_lit']]
subTeacherData = teacherdata[['distcd', 'tch_govt2', 'tch_govt3', 'tch_govt4', 'tch_govt5', 'tch_govt6', 'tch_govt7', 'tch_govt9', 'tch_pvt1', 'tch_pvt2', 'tch_pvt3', 'tch_pvt4', 'tch_pvt5', 'tch_pvt6', 'tch_pvt7', 'tch_pvt9']]

#Sort the data based on distcd
SubSchooldata.sort_values('distcd',ascending=True)
SubBasicdata.sort_values('distcd', ascending=True)
SubEnrolldata.sort_values('distcd', ascending=True)
subTeacherData.sort_values('distcd', ascending=True)

Mergedata = pd.concat([SubSchooldata,SubEnrolldata,subTeacherData,SubBasicdata],axis=1,join='inner')


In [0]:
## drop duplicate columns caused due to inner join
Mergedata = Mergedata.loc[:,~Mergedata.columns.duplicated()]
#Mergedata

In [0]:
## to drop all na columns
# Mergedata.dropna(inplace=True)
# Mergedata

Carry out the following steps to clean the data:

Overall_lit is our label. Without it we cannot meaningfully train.

So we can delete the rows with missing values in that column.
Bonus What else can we do with it?
Consider replacing the missing values in any other column

possibly with mean/median/mode.


In [0]:
Mergedata.isna().any()

In [0]:
## all columns that contain null values
nullcolumn = []
nullcolumn = Mergedata.columns[Mergedata.isnull().any()].tolist()
#nullcolumn

In [0]:
## number of nulls each column contains
na_col = []
list_na =[]
na_col = Mergedata.isnull().sum().tolist()
list_na = zip(nullcolumn,na_col)
#list_na

In [0]:
# for 
# Mergedata["schgovt4"].fillna(Mergedata["schgovt4"].mean())

In [0]:
for t in list_na:
  print(t[0])

In [0]:
## each null cells of the columns fill with mean of that particular column
for t in list_na:
  Mergedata[t[0]]=Mergedata[t[0]].fillna(Mergedata[t[0]].mean())
  


In [30]:
## to check that no null  columns are left
Mergedata.columns[Mergedata.isna().any()].tolist()

[]

In [141]:
Mergedata

,distcd,ac_year,schgovt1,schgovt2,schgovt3,schgovt4,schgovt5,schgovt6,schgovt7,schgovt9,...,Enr Pvt1,Enr Pvt2,Enr Pvt3,Enr Pvt4,Enr Pvt5,Enr Pvt6,Enr Pvt7,Enr Pvt9,totpopulation,overall_lit
0,3501,2012-13,61,26,19,0.000000,5,14.0,0.0,0.00000,...,2058.000000,1994.000000,5408.00000,0,0.000000,1153.000000,0.000000,0.00000,237586.0,2
1,3503,2012-13,104,29,5,3.000000,8,10.0,0.0,0.00000,...,779.000000,295.000000,19126.76003,0,0.000000,225.000000,0.000000,0.00000,105539.0,2
2,3502,2012-13,29,11,3,0.000000,6,7.0,0.0,0.00000,...,16.000000,29643.218608,19126.76003,0,8521.866969,9806.485261,5570.038549,2.59486,36819.0,2
3,2801,2012-13,3074,440,4,0.000000,1,97.0,430.0,0.00000,...,57987.000000,45282.000000,0.00000,0,0.000000,8169.000000,34569.000000,0.00000,2737738.0,0
4,2822,2012-13,2864,654,23,13.000000,495,1.0,50.0,0.00000,...,78173.000000,50979.000000,1704.00000,0,41617.000000,507.000000,6684.000000,0.00000,4083315.0,0
5,2823,2012-13,4234,528,21,0.000000,582,0.0,2.0,0.00000,...,89882.000000,38543.000000,3905.00000,0,50081.000000,484.000000,3557.000000,0.00000,4170468.0,1
6,2820,2012-13,2845,302,19,0.000000,361,0.0,7.0,1.00000,...,83877.000000,41090.000000,1321.00000,0,49146.000000,1419.000000,11333.000000,0.00000,2884524.0,1
7,2814,2012-13,3176,319,10,1.000000,0,73.0,532.0,0.00000,...,101920.000000,80477.000000,529.00000,0,304.000000,12728.000000,57451.000000,0.00000,5151549.0,1
8,2817,2012-13,2706,369,22,0.000000,379,0.0,0.0,0.00000,...,75736.000000,32198.000000,5001.00000,0,57450.000000,4101.000000,2668.000000,0.00000,4889230.0,1
9,2805,2012-13,621,10,2,5.000000,183,0.0,8.0,0.00000,...,233463.000000,64291.000000,36175.00000,0,145073.000000,4741.000000,8930.000000,0.00000,4010238.0,2


Use the functions below to adjust the outliers

smooth_out

takes a dataframe as input,
calculates the mean ( μ ) and standard deviation ( σ ) of every column
and replaces outliers with boundary values.
Outliers are those beyond  2σ  from  μ

In [0]:
import numpy as np

In [0]:
def clip_clamp(x, mean, sd):
  if x < mean - 2 * sd :
    return mean - 2 * sd
  elif x > mean + 2 * sd :
    return mean + 2 * sd
  else :
    return x
      

In [0]:
# Function to smooth the data
def smooth_out(Total_data):
    for i in Total_data.columns:
        if (isinstance(i, str)== False):
          mean = np.mean(Total_data[i].values, axis=0)
          sd = np.std(Total_data[i].values, axis=0)
          corrected = np.array([clip_clamp(x, mean, sd) for x in Total_data[i].values])
          Total_data[i] = pd.Series(corrected, index=Total_data[i].index)
    return Total_data

In [0]:
# np.array([1,2,3,4,5])
# total = pd.Series(np.array([1,2,3,4,5]),indes)

In [0]:
pd.Series?


In [30]:
smooth_out(Mergedata)

,distcd,schgovt1,schgovt2,schgovt3,schgovt4,schgovt5,schgovt6,schgovt7,schgovt9,schpvt1,...,tch_pvt1,tch_pvt2,tch_pvt3,tch_pvt4,tch_pvt5,tch_pvt6,tch_pvt7,tch_pvt9,totpopulation,overall_lit
0,3501,61,26,19,0.000000,5,14.0,0.0,0.00000,66,...,308,117,317,0,0,83,0,0,237586.0,2
1,3503,104,29,5,3.000000,8,10.0,0.0,0.00000,18,...,103,31,0,0,0,15,0,0,105539.0,2
2,3502,29,11,3,0.000000,6,7.0,0.0,0.00000,2,...,8,0,0,0,0,0,0,0,36819.0,2
3,2801,3074,440,4,0.000000,1,97.0,430.0,0.00000,275,...,2004,2298,0,0,0,327,2222,0,2737738.0,0
4,2822,2864,654,23,13.000000,495,1.0,50.0,0.00000,356,...,2063,2184,106,0,2307,0,41,0,4083315.0,0
5,2823,4234,528,21,0.000000,582,0.0,2.0,0.00000,440,...,2804,1909,225,0,3357,23,11,0,4170468.0,1
6,2820,2845,302,19,0.000000,361,0.0,7.0,1.00000,409,...,2010,1336,12,0,2320,9,102,0,2884524.0,1
7,2814,3176,319,10,1.000000,0,73.0,532.0,0.00000,556,...,3614,4347,55,0,12,469,3810,0,5151549.0,1
8,2817,2706,369,22,0.000000,379,0.0,0.0,0.00000,424,...,2210,1283,190,0,3007,53,113,0,4889230.0,1
9,2805,621,10,2,5.000000,183,0.0,8.0,0.00000,891,...,6984,2659,1371,0,8208,37,212,0,4010238.0,2



Use the function below (corr_features) to identify uncorrelated features and remove the remaining (dependent) features

corr_features takes pandas dataframe, columns in the dataframe and bar (correlation co-efficient)


In [0]:
# Function to find uncorrelated features
def corr_features(df, cols, bar=0.9):
  correlated = set()
  for pos, aCol in enumerate(cols[:-1]):
    if aCol in correlated: ## aCol has already been found dependent
      continue
    else: 
      for bCol in cols[pos + 1:]:
        score = df[aCol].corr(df[bCol])
        if -bar < score < bar:
          pass ## The correlation coefficient is within set limits
        else:
          correlated.add(bCol)
  return (set(cols) - correlated)

In [32]:
features = [x for x in corr_features(Mergedata, Mergedata.columns)]
uncorrelated = Mergedata[features]
uncorrelated


,Enr Govt3,schpvt5,schgovt2,schpvt4,tch_pvt9,tch_govt9,schpvt3,Enr Govt1,schpvt6,Enr Govt6,...,schgovt5,schgovt7,schgovt3,schgovt6,schpvt9,distcd,schgovt4,Enr Govt2,Enr Pvt3,overall_lit
0,10620,0.0,26,0.000000,0,0,8.0,3232,4,3739.000000,...,5,0.0,19,14.0,0.0,3501,0.000000,3359.0,5408.00000,2
1,1162,0.0,29,0.000000,0,0,0.0,3996,1,1625.000000,...,8,0.0,5,10.0,0.0,3503,3.000000,3808.0,19126.76003,2
2,357,0.0,11,0.000000,0,0,0.0,1510,0,838.000000,...,6,0.0,3,7.0,0.0,3502,0.000000,886.0,19126.76003,2
3,1266,0.0,440,0.000000,0,0,0.0,135664,27,23850.000000,...,1,430.0,4,97.0,0.0,2801,0.000000,53374.0,0.00000,0
4,6867,269.0,654,0.000000,0,0,5.0,145256,2,325.000000,...,495,50.0,23,1.0,0.0,2822,13.000000,71562.0,1704.00000,0
5,6305,379.0,528,0.000000,0,0,8.0,159102,2,0.000000,...,582,2.0,21,0.0,0.0,2823,0.000000,42118.0,3905.00000,1
6,4710,331.0,302,0.000000,0,0,1.0,114284,6,0.000000,...,361,7.0,19,0.0,0.0,2820,0.000000,23931.0,1321.00000,1
7,2812,1.0,319,0.000000,0,0,2.0,183269,25,13555.000000,...,0,532.0,10,73.0,0.0,2814,1.000000,43096.0,529.00000,1
8,6898,290.0,369,0.000000,0,0,10.0,150932,11,4060.962851,...,379,0.0,22,0.0,0.0,2817,0.000000,44585.0,5001.00000,1
9,1687,870.0,10,0.000000,0,0,53.0,71152,14,4060.962851,...,183,8.0,2,0.0,0.0,2805,5.000000,1587.0,36175.00000,2


Perform Mean Correction and Standard Scaling feature/column wise on the data

In [33]:
import sklearn
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
transformed = scaler.fit_transform(uncorrelated)

transformed

array([[ 1.00336882, -0.44083153, -0.69435541, ..., -0.55263814,
        -0.38425853,  1.05990127],
       [-0.21374395, -0.44083153, -0.68573324, ..., -0.54795217,
         0.        ,  1.05990127],
       [-0.31733623, -0.44083153, -0.73746622, ..., -0.57844751,
         0.        ,  1.05990127],
       ...,
       [ 0.10514011, -0.14327503, -0.76908081, ..., -0.58769421,
        -0.3873396 ,  1.05990127],
       [-0.2991915 , -0.38132023, -0.72884406, ..., -0.53903943,
        -0.41111984,  1.05990127],
       [-0.28413523, -0.44083153, -0.76908081, ..., -0.58769421,
         0.        , -1.48232569]])

Split the data into train and test sets

In [0]:
train_set = {}
test_set = {}

n_train = 0
n_test = 0


split = int(0.95*len(transformed))
train_set = transformed[:split]

test_set = transformed[split:]
n_train = len(train_set)
n_test = len(test_set)

In [0]:
train_label = Mergedata['overall_lit'].values[:split]
test_label = Mergedata['overall_lit'].values[split:]


Apply k-NN and Decision Tree classifiers on the preprocessed data and figure out which classifier gives the best result.

In [0]:
from sklearn.neighbors import KNeighborsClassifier

def KNN(k):
  neighbour = KNeighborsClassifier(k).fit(train_set,train_label)
  print(neighbour.predict(test_set))
  Accuracy = list(zip(neighbour.predict(test_set),test_label))
  x = len(Accuracy)
  correct = [l for l,m in Accuracy if l==m]
  y = len(correct)

  print(x,y)
  z = (y/x)*100
  print(z)
  print(neighbour.score(test_set, test_label))

In [60]:
KNN(3)

[1 1 1 0 0 0 1 0 1 0 1 2 1 1 2 1 1 1 0 1 1 0 1 0 1 0 0 0 0 1 1 1 1 2 2 2 2
 2 2 2 1 2 2 2 2 1 2 1 2 1 2 2 2 2 0 1 2 0 2 2 2 2 2 0 2 2 0]
67 55
82.08955223880598
0.8208955223880597


In [61]:
KNN(5)

[1 1 1 0 0 0 1 0 1 0 1 2 1 1 2 1 1 1 0 1 1 0 1 0 1 0 0 0 0 1 1 1 1 2 2 2 2
 2 2 2 1 2 2 2 2 1 2 1 2 1 1 2 1 1 1 1 2 0 2 1 2 2 2 1 2 2 0]
67 55
82.08955223880598
0.8208955223880597


In [63]:
KNN(31)

[1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 2 2 2
 2 2 2 1 2 2 2 2 1 2 2 1 1 1 2 2 2 1 2 2 1 1 1 1 1 1 1 2 1 0]
67 44
65.67164179104478
0.6567164179104478
